# Table of Contents:

[I. Load dữ liệu](#I.-Load-dữ-liệu)<br>
[1.1 Understand Data](#1.1-Understand-Data)<br>
[1.2 Prepare Data](#1.2-Prepare-Data)<br>
[1.3 Preprocessing data](#1.3-Preprocessing-data)<br>

[II. Build model](#II.-Build-model)<br>
[2.1 Baseline model](#2.1-Baseline-model)<br>
[2.2 Evaluation](#2.2-Evaluation)<br>

[III. Conclusions](#III.-Conclusions)

# I. Load dữ liệu

## 1.1 Understand Data

In [1]:
import numpy as np

In [2]:
from nltk.corpus import gutenberg

In [3]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [4]:
len(gutenberg.fileids())

18

Dữ liệu sắp sử dụng có **18** cuốn tiểu thuyết

In [5]:
lis = [] # initial list
for i in gutenberg.fileids():
    author_name = i.split('-')[0] # Tên tác giả
    if author_name not in lis:
        lis.append(author_name)
print(lis)
print('Có', len(lis), 'tác giả khác nhau.')

['austen', 'bible', 'blake', 'bryant', 'burgess', 'carroll', 'chesterton', 'edgeworth', 'melville', 'milton', 'shakespeare', 'whitman']
Có 12 tác giả khác nhau.


**Đọc thử một tác phẩm (Alice in Wonderland của Lewis Carroll)**

In [6]:
gutenberg.raw('carroll-alice.txt')[:300]

"[Alice's Adventures in Wonderland by Lewis Carroll 1865]\n\nCHAPTER I. Down the Rabbit-Hole\n\nAlice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into the\nbook her sister was reading, but it had no pictures or conversatio"

- Dạng đọc này chưa phải là plain text (cho người đọc) vì vẫn còn các dòng lệnh như \n (newline)
- Các từ dạng thuần văn nên độ nhiễu sẽ cao

**Thống kê đặc tính văn bản của 18 cuốn tiểu thuyết trên:**
- Cột 1 : số từ trong tác phẩm
- Cột 2 : số câu trong tác phẩm
- Cột 3 : lượng từ vựng trong tác phẩm
- Cột 4 : tên tác phẩm

In [7]:
for fileid in gutenberg.fileids():
    num_word = len(gutenberg.words(fileid)) # số từ
    num_sent = len(gutenberg.sents(fileid)) # số câu
    num_vocab = len(set(gutenberg.words(fileid))) # lượng từ vựng
    print(num_word, num_sent, num_vocab, fileid)

192427 7752 7811 austen-emma.txt
98171 3747 6132 austen-persuasion.txt
141576 4999 6833 austen-sense.txt
1010654 30103 13769 bible-kjv.txt
8354 438 1820 blake-poems.txt
55563 2863 4420 bryant-stories.txt
18963 1054 1764 burgess-busterbrown.txt
34110 1703 3016 carroll-alice.txt
96996 4779 8947 chesterton-ball.txt
86063 3806 8299 chesterton-brown.txt
69213 3742 6807 chesterton-thursday.txt
210663 10230 9593 edgeworth-parents.txt
260819 10059 19317 melville-moby_dick.txt
96825 1851 10751 milton-paradise.txt
25833 2163 3560 shakespeare-caesar.txt
37360 3106 5447 shakespeare-hamlet.txt
23140 1907 4017 shakespeare-macbeth.txt
154883 4250 14329 whitman-leaves.txt


## 1.2 Prepare Data

Cách lấy list từng đoạn văn

In [8]:
gutenberg.paras('carroll-alice.txt')

[[['[', 'Alice', "'", 's', 'Adventures', 'in', 'Wonderland', 'by', 'Lewis', 'Carroll', '1865', ']']], [['CHAPTER', 'I', '.'], ['Down', 'the', 'Rabbit', '-', 'Hole']], ...]

In [9]:
gutenberg.paras('carroll-alice.txt')[100]

[['The', 'Mouse', 'only', 'growled', 'in', 'reply', '.']]

Những `paras` này không phải là những câu văn mà là đoạn văn đã được lọc ra. Chứng minh:

In [10]:
len(gutenberg.paras('carroll-alice.txt'))

817

Độ dài của paras trong cuốn Alice in Wonderland được đo là 817 nhưng với phần thống kê ở trước là có **1703** câu.

**Ý tưởng**: lập một dataframe với **2** cột, một là đoạn văn và một là tên tác giả.

In [11]:
# get the data
from itertools import chain
author_column = []
paragraphs_column = []
for fileid in gutenberg.fileids():
    author_name = fileid.split('-')[0] # Tên tác giả
    work = gutenberg.paras(fileid) # nhiều đoạn văn trong fileid đó
    for paragraph in work:
        author_column.append(author_name) # mỗi đoạn tương ứng với tên của tác giả (bao nhiêu đoạn bấy nhiêu lần tên tác giả)
        paragraph = list(chain.from_iterable(paragraph)) # để bỏ bớt một list chồng
        paragraphs_column.append(paragraph)

In [12]:
len(author_column)

47887

In [13]:
len(paragraphs_column)

47887

Ok vậy là ta có **2 cột**. Và biết được trong 18 cuốn tiểu thuyết ở trên thì có tổng cộng 47887 đoạn văn. Giờ ta sẽ lập dataframe cho **2 cột** này.

In [14]:
import pandas as pd
dataframe = pd.DataFrame({'sentence': paragraphs_column,
                         'label': author_column})
dataframe.head()

,sentence,label
0,"[[, Emma, by, Jane, Austen, 1816, ]]",austen
1,"[VOLUME, I]",austen
2,"[CHAPTER, I]",austen
3,"[Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",austen
4,"[She, was, the, youngest, of, the, two, daught...",austen


In [15]:
dataframe['sentence'][0] # xem câu đầu tiên trong cột 'sentence'

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']']

## 1.3 Preprocessing data

- Nối các từ thành lại 1 câu
- Giờ ta cần xử lý để bỏ những stop words (như but, and, than...)
- Bỏ những dấu , . ! ?
- Biến chữ hoa thành chữ thường

**Nối các từ lại thành câu**

In [16]:
dataframe['sentence'] = [' '.join(sentence) for sentence in dataframe['sentence']] # list comprehension

In [17]:
dataframe.head()

,sentence,label
0,[ Emma by Jane Austen 1816 ],austen
1,VOLUME I,austen
2,CHAPTER I,austen
3,"Emma Woodhouse , handsome , clever , and rich ...",austen
4,She was the youngest of the two daughters of a...,austen


**Tiền xử lý**

In [18]:
from gensim.parsing.preprocessing import preprocess_string
#from gensim.parsing.preprocessing import strip_tags # bỏ tag name 
from gensim.parsing.preprocessing import remove_stopwords # bỏ stop words
from gensim.parsing.preprocessing import strip_punctuation # bỏ dấu
CUSTOM_FILTERS = [lambda x: x.lower(), strip_punctuation, remove_stopwords]

Lưu ý trong 'CUSTOM_FILTERS' ở đây ta chỉ muốn biến chữ hoa về chữ thường, bỏ hệ thống dấu trong câu văn, và bỏ stop words, bỏ những đoạn có dưới 3 từ.

In [19]:
dataframe['sentence'] = [preprocess_string(sentence, CUSTOM_FILTERS) for sentence in dataframe['sentence']]

In [20]:
dataframe.head()

,sentence,label
0,"[emma, jane, austen, 1816]",austen
1,[volume],austen
2,[chapter],austen
3,"[emma, woodhouse, handsome, clever, rich, comf...",austen
4,"[youngest, daughters, affectionate, indulgent,...",austen


Ta có thể thấy có những đoạn văn có lượng từ rất nhỏ như ở vị trí hàng 0,1,2. Ta drop những hàng có độ dài đoạn văn nhỏ hơn **5**

In [ ]:
lis = [] # tìm index của các element có độ dài dưới 5
for i in range(len(dataframe)):
    n= dataframe['sentence'].iloc[i]
    if len(n) < 5:
        lis.append(i)     

In [29]:
dataframe = dataframe.drop(index=lis) # drop những hàng đó đi
dataframe.head()

,sentence,label
3,"[emma, woodhouse, handsome, clever, rich, comf...",austen
4,"[youngest, daughters, affectionate, indulgent,...",austen
5,"[sixteen, years, miss, taylor, mr, woodhouse, ...",austen
6,"[real, evils, emma, s, situation, power, havin...",austen
7,"[sorrow, came, gentle, sorrow, shape, disagree...",austen


In [30]:
dataframe['label'].value_counts()

bible          24383
austen          4574
chesterton      3506
edgeworth       3182
melville        2328
whitman         2065
shakespeare     1598
bryant           970
carroll          673
burgess          225
blake            224
milton            16
Name: label, dtype: int64

Dữ liệu lúc này khá imbalance. Dữ liệu imbalance không hẳn là một vấn đề lớn trong deep learning nhưng ta có thể suy tính bằng một cách khác như dùng thuật toán không bị ảnh hưởng của sự mất cân bằng dữ liệu như: random forest, logistic regression

**Dùng Word2vec để vector hóa từng chữ**

In [31]:
len(dataframe['sentence'])

43744

In [32]:
from gensim.models import Word2Vec
model = Word2Vec(dataframe['sentence'], size=150, window=5, min_count=1, seed=0, compute_loss=True)

- Size là số chiều khi embedding, mặc định là 100
- Window là khoảng cách tối đa giữa từ mục tiêu và các từ xung quanh nó
- Min count là số lượng đếm tối thiểu các từ khi training, từ có số lượng nhỏ hơn sẽ bị lờ đi. Mặc định là 5
- Workers là số lượng chia cắt trong khi training. Mặc định là 3
- sg là thuật toán train, 'cbow' là 0, 'skip gram' là 1. Mặc định CBOW

In [33]:
model.train(dataframe['sentence'], total_examples=len(dataframe['sentence']), epochs=10)

(9140472, 9609340)

In [34]:
print('Latest training loss: %.2f'%model.get_latest_training_loss())

Latest training loss: 0.00


Thử xem những từ gần với từ 'alice' trong vocab của chúng ta.

In [36]:
model.wv.most_similar('alice', topn=6)

[('hatter', 0.8703389167785645),
 ('gryphon', 0.8376980423927307),
 ('duchess', 0.8369855284690857),
 ('dormouse', 0.8337677717208862),
 ('laughing', 0.8059630393981934),
 ('smiling', 0.7840709686279297)]

In [37]:
paragraphs_embedding = []

for paragraph in dataframe['sentence']:
    paragraph_embedding = []
    for word in paragraph:
        word_embedding = model.wv.get_vector(word)
        paragraph_embedding.append(word_embedding)
    paragraphs_embedding.append(paragraph_embedding)

Nối các vector lại trong đoạn văn

In [38]:
# trim and pad embeddign paragraph
from keras.preprocessing import sequence
paragraphs_embedding = sequence.pad_sequences(paragraphs_embedding, maxlen=130, padding='post', truncating='post', value=0.0)

Using TensorFlow backend.


- khong noi vector: np.mean(paragraph_embedding)
- kich thuoc w2v co dinh 150, nen mean cua cac vector co chieu giong nhau
- dung cac thuat toan khac nhu regression, random forest
- random forest solve duoc cho imbalance data

In [39]:
model.wv.get_vector('wonderland').shape

(150,)

chiều vector của các từ trong vocab là như nhau. Như đã set trong gensim là **150**

In [40]:
paragraphs_embedding.shape

(43744, 130, 150)

In [41]:
one_hot = pd.get_dummies(dataframe['label'])
one_hot.shape

(43744, 12)

In [42]:
feature_name= list(one_hot.columns)
print(feature_name)

['austen', 'bible', 'blake', 'bryant', 'burgess', 'carroll', 'chesterton', 'edgeworth', 'melville', 'milton', 'shakespeare', 'whitman']


In [43]:
#X = df
y = one_hot
X = paragraphs_embedding

**Hold out**

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
X_train.shape

(34995, 130, 150)

In [33]:
#del dataframe

# II. Build model

In [46]:
# CNN for sequence classification
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D

## 2.1 Baseline model

In [47]:
# CONVOLUTIONAL NEURAL NETWORK
cnn = Sequential()
cnn.add(Conv1D(150, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(X.shape[1], X.shape[2])))
cnn.add(Conv1D(100, kernel_size=3, strides=1, padding='same', activation='relu'))
cnn.add(Flatten())
cnn.add(Dropout(0.5,seed=0))
cnn.add(Dense(150))
cnn.add(Dense(12, activation='softmax'))
# compile model
cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(cnn.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 130, 150)          67650     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 130, 100)          45100     
_________________________________________________________________
flatten_1 (Flatten)          (None, 13000)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 150)               1950150   
_________________________________________________________________
dense_2 (Dense)      

In [48]:
cnn.fit(X_train, y_train, epochs=30, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
34995/34995 [==============================] - 9s 262us/step - loss: 0.5487 - acc: 0.8144
Epoch 2/30
34995/34995 [==============================] - 7s 188us/step - loss: 0.3373 - acc: 0.8859
Epoch 3/30
34995/34995 [==============================] - 7s 186us/step - loss: 0.2830 - acc: 0.9038
Epoch 4/30
34995/34995 [==============================] - 7s 188us/step - loss: 0.2444 - acc: 0.9151
Epoch 5/30
34995/34995 [==============================] - 7s 187us/step - loss: 0.2156 - acc: 0.9244
Epoch 6/30
34995/34995 [==============================] - 7s 187us/step - loss: 0.1890 - acc: 0.9333
Epoch 7/30
34995/34995 [==============================] - 7s 186us/step - loss: 0.1712 - acc: 0.9412
Epoch 8/30
34995/34995 [==============================] - 7s 187us/step - loss: 0.1592 - acc: 0.9442
Epoch 9/30
34995/34995 [==============================] - 7s 187us/step - loss: 0.1470 - acc: 0.9503
Epoch 10/30
34995/34995 [==================

In [50]:
cnn.save('D:/CBD robotics course/Assignment/Assignment 09/trained_model.h5')

## 2.2 Evaluation

In [49]:
# Final evaluation of the model
scores = cnn.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 90.42%


Đánh giá qua classification_report để có cái nhìn insight hơn về kết quả

In [51]:
from sklearn.metrics import classification_report
predicted_y = cnn.predict(X_test)

- Ta có `predicted_y` là một numpy array, ta cần `np.argmax()` để tìm ra index thực của (cũng là nhãn) của dữ liệu
- Trong khi đầu vào y của chúng ta là một dataframe nên `y_test` cũng là một dataframe

In [52]:
feature_name= list(one_hot.columns) # list các nhãn là chữ
y_pred = [np.argmax(i) for i in predicted_y] # Tìm argmax
y_pred_label = [feature_name[i] for i in y_pred] # Biến list các nhãn này về chữ

y_true_label = [np.argmax(y_test.iloc[i]) for i in range(y_test.shape[0])] # np.argmax(y_test.iloc[i] trả về
                                                                           # column name trong dataframe
# classification report
print(classification_report(y_true_label, y_pred_label))

C:\Users\OS\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


              precision    recall  f1-score   support

      austen       0.93      0.88      0.91       953
       bible       1.00      1.00      1.00      4795
       blake       0.54      0.17      0.26        41
      bryant       0.60      0.56      0.58       199
     burgess       1.00      0.76      0.86        45
     carroll       0.85      0.62      0.72       143
  chesterton       0.78      0.77      0.78       715
   edgeworth       0.73      0.75      0.74       624
    melville       0.75      0.78      0.77       473
      milton       0.00      0.00      0.00         3
 shakespeare       0.96      0.86      0.91       339
     whitman       0.63      0.85      0.73       419

    accuracy                           0.90      8749
   macro avg       0.73      0.67      0.69      8749
weighted avg       0.91      0.90      0.90      8749



C:\Users\OS\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [53]:
y_test.head()

,austen,bible,blake,bryant,burgess,carroll,chesterton,edgeworth,melville,milton,shakespeare,whitman
6219,0,1,0,0,0,0,0,0,0,0,0,0
8978,0,1,0,0,0,0,0,0,0,0,0,0
19067,0,1,0,0,0,0,0,0,0,0,0,0
22999,0,1,0,0,0,0,0,0,0,0,0,0
13207,0,1,0,0,0,0,0,0,0,0,0,0


In [54]:
y_test.iloc[0]

austen         0
bible          1
blake          0
bryant         0
burgess        0
carroll        0
chesterton     0
edgeworth      0
melville       0
milton         0
shakespeare    0
whitman        0
Name: 6219, dtype: uint8

Ta thấy mô hình phân loại rất tệ với tác giả milton, blake. Ta mở xem thử những tác phẩm của tác giả này là về thể loại gì.

In [4]:
print(gutenberg.raw('blake-poems.txt')[:300])

[Poems by William Blake 1789]

 
SONGS OF INNOCENCE AND OF EXPERIENCE
and THE BOOK of THEL


 SONGS OF INNOCENCE
 
 
 INTRODUCTION
 
 Piping down the valleys wild,
   Piping songs of pleasant glee,
 On a cloud I saw a child,
   And he laughing said to me:
 
 "Pipe a song about a Lamb!"
   So I piped


In [3]:
print(gutenberg.raw('milton-paradise.txt')[:300])

[Paradise Lost by John Milton 1667] 
 
 
Book I 
 
 
Of Man's first disobedience, and the fruit 
Of that forbidden tree whose mortal taste 
Brought death into the World, and all our woe, 
With loss of Eden, till one greater Man 
Restore us, and regain the blissful seat, 
Sing, Heavenly Muse, that, o


Vậy là 2 tác phẩm của 2 nhà văn này là về thơ. Khác biệt với thể loại văn xuôi/viết so với các tác giả còn lại. Điều đó cho ta đầu mối rằng các tác phẩm có cùng một thể loại (văn xuối, hay thơ) thì nhận dạng tốt hơn.

# III. Conclusions
- Dữ liệu dạng text cần phải qua một quá trình tiền xử lý mới có thể sử dụng được. Quá trình này có thể tóm tắt như sau: Load dữ liệu -> những dữ liệu này có phải cùng loại (thơ và văn xuôi không xếp chung chẳng hạn) -> bỏ dấu, biến về chữ thường, loại bỏ stop word, tách ra từng đoạn, bỏ những đoạn quá ngắn -> vector hóa, nối vector hoặc np.mean tất cả các vector trong cùng môt đoạn -> bỏ vào mô hình.
- Việc lập nên dataframe giúp ý tưởng trên nên rõ ràng hơn, đồng thời cũng mô tả được dữ liệu của ta như thế nào (liệu có imbalance không)
- Mô hình liệu train bao nhiêu epoch thì dừng được (mới thử đến epochs thứ 30)? Những dữ liệu dạng text khi nào dùng cross validation thì phù hợp?
- Nếu không dùng mô hình deeplearning, liệu ý tưởng dùng mô hình classification như random forest, logistic regression liệu có tốt hơn? Cũng như đỡ tốn 'tài nguyên' hơn?